In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ba1b74964435ddd921d6e1162865677a8a102941eb94b447676000d04c1eb57e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [ ]:
import pyspark
import pyspark.sql  as pyspark_sql
import pyspark.sql.types as pyspark_types
import pyspark.sql.functions  as pyspark_functions
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark_sql.SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, dense_rank,desc
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder.appName("HighEarners").getOrCreate()

# Sample data for Employee DataFrame
employee_data = [
    (1, "Joe", 85000, 1),
    (2, "Henry", 80000, 2),
    (3, "Sam", 60000, 2),
    (4, "Max", 90000, 1),
    (5, "Janet", 69000, 1),
    (6, "Randy", 85000, 1),
    (7, "Will", 70000, 1)
]

# Sample data for Department DataFrame
department_data = [
    (1, "IT"),
    (2, "Sales")
]

# Define schema for Employee and Department DataFrames
employee_columns = ["id", "name", "salary", "departmentId"]
department_columns = ["id", "name"]

# Create DataFrames
employee_df = spark.createDataFrame(employee_data, employee_columns)
department_df = spark.createDataFrame(department_data, department_columns)

In [ ]:
employee_df.show()
department_df.show()

+---+-----+------+------------+
| id| name|salary|departmentId|
+---+-----+------+------------+
|  1|  Joe| 85000|           1|
|  2|Henry| 80000|           2|
|  3|  Sam| 60000|           2|
|  4|  Max| 90000|           1|
|  5|Janet| 69000|           1|
|  6|Randy| 85000|           1|
|  7| Will| 70000|           1|
+---+-----+------+------------+

+---+-----+
| id| name|
+---+-----+
|  1|   IT|
|  2|Sales|
+---+-----+



In [ ]:
# row_number() example
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
df=employee_df.join(department_df,employee_df.departmentId==department_df.id)
df.show()
windowSpec  = Window.partitionBy("departmentId").orderBy(desc("salary"))
df=df.withColumn("dense_rank",dense_rank().over(windowSpec))
df=df.filter(col('dense_rank')<4)


+---+-----+------+------------+---+-----+
| id| name|salary|departmentId| id| name|
+---+-----+------+------------+---+-----+
|  1|  Joe| 85000|           1|  1|   IT|
|  4|  Max| 90000|           1|  1|   IT|
|  5|Janet| 69000|           1|  1|   IT|
|  6|Randy| 85000|           1|  1|   IT|
|  7| Will| 70000|           1|  1|   IT|
|  2|Henry| 80000|           2|  2|Sales|
|  3|  Sam| 60000|           2|  2|Sales|
+---+-----+------+------------+---+-----+



In [ ]:
df.show()

+---+-----+------+------------+---+-----+----------+
| id| name|salary|departmentId| id| name|dense_rank|
+---+-----+------+------------+---+-----+----------+
|  4|  Max| 90000|           1|  1|   IT|         1|
|  1|  Joe| 85000|           1|  1|   IT|         2|
|  6|Randy| 85000|           1|  1|   IT|         2|
|  7| Will| 70000|           1|  1|   IT|         3|
|  2|Henry| 80000|           2|  2|Sales|         1|
|  3|  Sam| 60000|           2|  2|Sales|         2|
+---+-----+------+------------+---+-----+----------+

